In [4]:
# Team: Tethical Machine Learning
# Team members: Mellanie Martin, Wyatt Pigeon, Koby Grah

import pandas as pd
import numpy as np

def main():
    data = pd.read_csv("brist1d/train.csv")

    columns_to_combine = [col for col in data.columns if col.startswith('bg-')]
    drop = [col for col in data.columns if (col.startswith('carbs-') or col.startswith('hr-')  or col.startswith('steps-')  or col.startswith('activity-'))]

    combined_columns = []
    for i in range(0, len(columns_to_combine), 3):  # Every three columns
        group = columns_to_combine[i:i+3]  # Group 'em up
    
        if len(group) == 3:  # If actually got 3 columns
            new_column_name = f"{group[0]}-{group[2][-5:]}"  # Rename column group with first time and last time
            data[new_column_name] = data[group].bfill(axis=1).iloc[:, 0]  # Fill new column with whichever column actually has a value
            combined_columns.append(new_column_name)  # Add new column to combined list

    data = data.drop(columns=columns_to_combine)  # Drop original columns
    data = data.drop(columns=drop)  # If you need to drop anything for visibility
    desired_columns = ('bg-', 'insulin-', 'cals-')
    
    xs = [col for col in data.columns if col.startswith(desired_columns)]
    ys = data['bg+1:00']

#    print(xs)
#    print(ys)
    
#    display(data)

    

if __name__ == "__main__":
    main()

/var/folders/n9/v_rk5kln2ljbl30lc2gg7nnw0000gn/T/ipykernel_23896/903420183.py:8: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("brist1d/train.csv")


0         13.4
1         12.8
2         15.5
3         14.8
4         12.7
          ... 
177019    11.1
177020    10.9
177021    10.7
177022    10.5
177023    10.2
Name: bg+1:00, Length: 177024, dtype: float64


Notes:
1. Final predicted value is a float, so this is a regression problem
2. Potential models to look at; Kernel Ridge, AdaBoost, Gradient Boosting
3. Pipeline with minimum 3 stages is needed, and we need to use GridSearchCV